In [3]:
from JuliaSet import checktick
import time
from timeit import default_timer as timer


def timetime():
    return time.time() * 1e9 # convert secound to nanosecound

def time_it():
    return timer() * 1e9

def time_ns():
    return time.time_ns() 


time_granularity = checktick(timetime) # conver secound to nanosecond
timeit_granularity = checktick(time_it)
time_ns_granularity = checktick(time_ns)

print("time_granularity (time.time): ", time_granularity, "ns")
print("timeit_granularity (timeit.default_timer): ", timeit_granularity, "ns")
print("time_ns_granularity (time.time_ns): ", time_ns_granularity, "ns")




time_granularity (time.time):  416768.0 ns
timeit_granularity (timeit.default_timer):  300.0 ns
time_ns_granularity (time.time_ns):  552704.0 ns


Task 1.2

Develop the timer decorator

In [7]:
import time
from functools import wraps
import numpy as np

def checktick(t_f):
   M = 200
   timesfound = np.empty((M,))
   for i in range(M):
      t1 = t_f()  # get timestamp from timer
      t2 = t_f() # get timestamp from timer
      while (t2 - t1) < 1e-16: # if zero then we are below clock granularity, retake timing
          t2 = t_f() # get timestamp from timer
      t1 = t2 # this is outside the loop
      timesfound[i] = t1 # record the time stamp
   minDelta = 1000000
   Delta = np.diff(timesfound) # it should be cast to int only when needed
   minDelta = Delta.min()
   return minDelta


# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

# decorator to timeit
def timeit(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = timer()
        result = fn(*args, **kwargs)
        t2 = timer()
        print(f"@timefn: {fn.__name__} took {t2 - t1} seconds")
        return result
    return measure_time

@timeit
def calc_pure_python(desired_width, max_iterations):
    """Create a list of complex coordinates (zs) and complex parameters (cs),
    build Julia set"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of coordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our
    # function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    # This sum is expected for a 1000^2 grid with 300 iterations
    # It ensures that our code evolves exactly as we'd intended
    assert sum(output) == 33219980
@timeit
def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

if __name__ == "__main__":
    # Calculate the Julia set using a pure Python solution with
    # reasonable defaults for a laptop
    calc_pure_python(desired_width=1000, max_iterations=300) 

Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 6.91051379999999 seconds
calculate_z_serial_purepython took 6.910264492034912 seconds
@timefn: calc_pure_python took 7.439861500000006 seconds


Raport averages and standard deviation 

In [11]:
from JuliaSet import calc_pure_python, calculate_z_serial_purepython, x1, x2, y1, y2, c_real, c_imag
import numpy as np
import time



def profile_function(func, *args, **kwargs):
    times = []
    runs = 10
    for _ in range(runs):
        start = time.perf_counter_ns()
        func(*args, **kwargs)
        end = time.perf_counter_ns()
        times.append(end - start)

    average_time = np.mean(times)
    std_deviation = np.std(times)
    return average_time, std_deviation

if __name__ == "__main__":
    desired_width = 1000
    max_iterations = 300
    print("Profiling calc_pure_python:")
    avg, std_dev = profile_function(calc_pure_python, desired_width, max_iterations)
    print(f"Average time: {avg / 1e9:.6f} seconds")
    print(f"Standard deviation: {std_dev / 1e9:.6f} seconds\n")

    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = [x1 + i * x_step for i in range(desired_width)]
    y = [y2 + j * y_step for j in range(desired_width)]
    zs = [complex(xc, yc) for yc in y for xc in x]
    cs = [complex(c_real, c_imag)] * len(zs)
    
    print("Profiling calculate_z_serial_purepython:")
    avg, std_dev = profile_function(calculate_z_serial_purepython, max_iterations, zs, cs)
    print(f"Average time: {avg / 1e9:.6f} seconds")
    print(f"Standard deviation: {std_dev / 1e9:.6f} seconds")

Profiling calc_pure_python:
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.742023706436157 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.928879022598267 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.886752367019653 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.043401479721069 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 8.145570993423462 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 9.094084978103638 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.388136148452759 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.949799537658691 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 6.945314884185791 seconds
Length of x: 1000
Total elements: 1000000
cal